# Functional Reactive Programming

In functional view, we can take sequence of events and aggregate into a signal.
Imperative reactive programming is about reacting to sequences of events that happen in time.

**Functional view:** Aggregate an event sequence into a signal.
* A signal is a value that changes over time.
* It is represented as a function from time to the value domain.
* Instead of propagating updates to mutable state, we define new
signals in terms of existing ones.

**Event-based view:**

Whenever the mouse moves, an event
```scala
MouseMoved(toPos: Position)
```
is fired.

**FRP view:**

A signal, 
```scala
mousePosition: Signal[Position] 
```
which at any point in time represents the current mouse position.

In event-based view whenever mouse moved then application gets `MouseMoved` events get fired and it's position is updated. All of the updates are imperative.

In functional point of view, the core idea is that I define a signal, call it also `mousePosition`, which is now a signal of position, which at any point in time represents the current mouse position. So, it's a function from the domain of time values to positions.

Functional Reactive Programming started in 1997 with the paper *Functional Reactive Animation by Conal Elliot and Paul Hudak, and Conal* also wrote a, a language called Fran, which was implemented as an embedded library in Haskell.

There have been many FRP systems since, both standalone languages and embedded libraries.

Some examples are: Flapjax, Elm, Bacon.js, React4J.

Event streaming dataflow programming systems such as Rx (which we will see in two weeks), are related but the term FRP is not commonly used for them.
We will introduce FRP by means of of a minimal class, `frp.Signal` whose implementation is explained at the end of this module.
`frp.Signal` is modelled after `Scala.react`, which is described in the paper *Deprecating the Observer Pattern.*

## Fundamental Signal Operations

There are two fundamental operations over signals:

1. Obtain the value of the signal at the current time.

In our library this is expressed by `()` application.
```scala
mousePosition() // the current mouse position
```
2. Define a signal in terms of other signals.

In our library, this is expressed by the Signal constructor.
```scala
def inReactangle(LL: Position, UR: Position): Signal[Boolean] =
Signal {
val pos = mousePosition()
LL <= pos && pos <= UR
}
```
3. Constant Signal

The `Signal(...)` syntax can also be used to define a signal that has always the same value:
```scala
val sig = Signal(3)
```

## Variable Signals
* Values of type Signal are immutable.

* But our library also defines a subclass Var of Signal for signals that can be changed.

* `Var` class provides an `update` operation, which allows to redefine the value of a signal from the current time on.

```scala
val sig = Var(3)
sig.update(5)
```

In Scala, calls to update can be written as assignments.

For instance, for an array `arr`
```scala
arr(i) = 0

```
is translated to
```scala
arr.update(i, 0)
```
which calls an update method which can be thought of as follows:
```
class Array[T] {
def update(idx: Int, value: T): Unit
...
}
```

Generally, an indexed assignment like $f(E1, ..., En) = E$ is translated to $f.update(E1, ..., En, E)$.
This works also if $n = 0$: $f() = E$ is shorthand for $f.update(E).$
Hence,
```scala
sig.update(5)
```
can be abbreviated to

```scala
sig() = 5
```

Signals of type Var look a bit like mutable variables, where
```scala
sig()
```
is dereferencing, and
```scala
sig() = newValue
```
is update.

But there’s a crucial difference:

We can map over signals, which gives us a relation between two signals that is maintained automatically, at all future points in time.
No such mechanism exists for mutable variables; we have to propagate all updates manually

## Example

Repeat the `BankAccount` example of last section with signals.

Add a signal `balance` to BankAccounts.

Define a function consolidated which produces the sum of all balances of a given list of accounts.

What savings were possible compared to the `publish/subscribe` implementation?

Let's implement `BankAccount` in functional reactive style.


In [1]:
import Signal.caller
import scala.util.DynamicVariable

class Signal[T](expr: => T) {
  import Signal._
  private var myExpr: () => T = _
  private var myValue: T = _
  private var observers: Set[Signal[_]] = Set()
  update(expr)
  
  //update method gets called during the initialization of the Signal or
  //someone calls an update operation on a Var, or the value of a dependent
  ///signal changes. 
  //"protected" means only subclasses of Signal has access to this method
  //clients of the Signal cannot. This means clients of Signal cannot call update
  protected def update(expr: => T): Unit = {
    myExpr = () => expr
    computeValue()
  }
  protected def computeValue(): Unit = {
    val newValue = caller.withValue(this)(myExpr())
    //re-evaluating the callers
    if (myValue != newValue) {
      myValue = newValue
      val obs = observers
      observers = Set() //clear the observers set
      obs.foreach(_.computeValue()) //add the updated observer into the observer sets
    }
  }
  
  
  //catches cyclic signal s() = s() + 1, If you don't add this assert, then infinite 
  //recursion and stackoverflow
  def apply(): T = {
    observers += caller.value
    assert(!caller.value.observers.contains(this), "cyclic signal definition")
    myValue
  }
}
object NoSignal extends Signal[Nothing](???) {
  //below code disables computeValue because we can't evaluate an expression of type Nothing
  override def computeValue() = () //empty expression can't evaluate NoSignal
}
object Signal {

  import scala.util.DynamicVariable
  
  //private val caller = new StackableVariable[Signal[_]](NoSignal)    // <--global variable
  
  /* global variable caller! Global variables in concurrency is bad idea (results in 
   * race conditions). One way to do that is to use synchronization, which comes
   * with its own problems (use of threads could create deadlock). We replace the 
   * "new StackableVariable" below by "new DynamicVariable" in 
   * scala.util.DynamicVariable to replace global state by thread-local state (each
   * thread accesses a separate copy of a variable.
   * 
   */
  private val caller = new DynamicVariable[Signal[_]](NoSignal)
  
  def apply[T](expr: => T) = new Signal(expr)
}

//Var is a sub-class of Signal
class Var[T](expr: => T) extends Signal[T](expr) {
  //client of Var can call update. That's why here, the keyword "protected" is overridden.
  override def update(expr: => T): Unit = super.update(expr)
}
object Var {
  def apply[T](expr: => T) = new Var(expr) //constructor
}

  //global variables - don't use if doing concurrency
class StackableVariable[T](init: T) {
  private var values: List[T] = List(init)
  def value: T = values.head
  def withValue[R](newValue: T)(op: => R): R = {
    values = newValue :: values
    try op finally values = values.tail
  }
}

import Signal.caller

import scala.util.DynamicVariable


defined class Signal
defined object NoSignal
defined object Signal
defined class Var
defined object Var
defined class StackableVariable

In [2]:
class BankAccountSignal {
  
  val balance = Var(0) //balance is a variable signal
  
  def deposit(amount: Int): Unit = {
    if (amount > 0) {
      val b = balance()
      balance() = b + amount
    }
  }
  
  def withdraw(amount: Int): Unit = {
    if (0 < amount && amount <= balance()) {
      val b = balance()
      balance() = b - amount
    } else throw new Error("insufficient funds")
  }

}

def consolidated(accts: List[BankAccountSignal]): Signal[Int] = Signal{
    accts.map(_.balance()).sum
}

defined class BankAccountSignal
defined function consolidated

In [3]:
val a,b = new BankAccountSignal                                                           
val c = consolidated(List(a,b))   

a: BankAccountSignal = ammonite.$sess.cmd1$Helper$BankAccountSignal@1e1c9fe
b: BankAccountSignal = ammonite.$sess.cmd1$Helper$BankAccountSignal@10c83d4
c: Signal[Int] = ammonite.$sess.cmd0$Helper$Signal@46529e

In [4]:
c()

res3: Int = 0

In [5]:
a deposit 20

In [6]:
c()

res5: Int = 20

In [7]:
b deposit 30

In [8]:
c()

res7: Int = 50

Say, we want to have another signal exchange rate.

In [10]:
val xchange = Signal(246.0)
val inDollar = Signal(c()*xchange())

xchange: Signal[Double] = ammonite.$sess.cmd0$Helper$Signal@13e3806
inDollar: Signal[Double] = ammonite.$sess.cmd0$Helper$Signal@4ee39e

In [11]:
inDollar()

res10: Double = 12300.0

In [12]:
b withdraw 10

In [13]:
inDollar()

res12: Double = 9840.0

That's bank account done redone with signals. It's much shorter and cleaner. 
Note that there’s an important difference between the variable assignment

`v = v + 1`
and the signal update

`s() = s() + 1`

In the first case, the new value of `v` becomes the `_old+ value of v plus 1`.

In the second case, we try define a signal `s` to be at all points in time one
larger than itself.
This obviously makes no sense!

Consider the two code fragments below

```scala
val num = Signal(1) val twice = Signal(num() * 2) num() = 2
```

```scala 
var num = Signal(1) val twice = Signal(num() * 2) num = Signal(2)
```

So they yield the same final value for twice()?

In second case num points to new signal whose value is 2. While the first always stays 2.